In [3]:
import os
from api_key import OPENAI_API_KEY
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import gradio as gr


In [9]:

# 初始化 embedding 和加载已保存向量库
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)


In [10]:

# 构建问答系统
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)


In [11]:

# 回答函数
def answer(query):
    result = qa(query)
    sources = "\n".join([f"- {doc.metadata.get('source', '未知来源')}: {doc.page_content[:150].strip()}…" for doc in result["source_documents"]])
    return result["result"] + "\n\n**Sources:**\n" + sources



In [12]:
# Gradio 界面
iface = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
    outputs="markdown",
    title="知识库问答助手",
    description="基于本地 FAISS 向量库的文档问答系统。",
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\user\AppData\Local\Temp\ipykernel_26732\3348745992.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(query)
